# Lab | Agent & Vector store

**Change the state union dataset and replicate this lab by updating the prompts accordingly.**

One such dataset is the [sonnets.txt](https://github.com/martin-gorner/tensorflow-rnn-shakespeare/blob/master/shakespeare/sonnets.txt) dataset or any other data of your choice from the same git.

# Combine agents and vector stores

This notebook covers how to combine agents and vector stores. The use case for this is that you've ingested your data into a vector store and want to interact with it in an agentic manner.

The recommended method for doing so is to create a `RetrievalQA` and then use that as a tool in the overall agent. Let's take a look at doing this below. You can do this with multiple different vector DBs, and use the agent as a way to route between them. There are two different ways of doing this - you can either let the agent use the vector stores as normal tools, or you can set `return_direct=True` to really just use the agent as a router.

## Create the vector store

In [1]:
!pip install chromadb langchain langchain_community langchain_openai

In [2]:
from langchain.chains import RetrievalQA
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAI, OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader

In [5]:
llm = OpenAI(temperature=0)

In [6]:
from pathlib import Path

relevant_parts = []
for p in Path(".").absolute().parts:
    relevant_parts.append(p)
    if relevant_parts[-3:] == ["langchain", "docs", "modules"]:
        break
doc_path = str(Path(*relevant_parts) / "sonnets.txt")

In [7]:
loader = TextLoader(doc_path)
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=50)
texts = text_splitter.split_documents(documents)

embeddings = OpenAIEmbeddings()
docsearch = Chroma.from_documents(texts, embeddings, collection_name="sonnets")

In [8]:
state_of_union = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=docsearch.as_retriever()
)

In [9]:
from langchain_community.document_loaders import WebBaseLoader

In [10]:
loader = WebBaseLoader("https://beta.ruff.rs/docs/faq/")

In [11]:
docs = loader.load()
ruff_texts = text_splitter.split_documents(docs)
ruff_db = Chroma.from_documents(ruff_texts, embeddings, collection_name="ruff")
ruff = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=ruff_db.as_retriever()
)

## Create the Agent

In [12]:
# Import things that are needed generically
from langchain.agents import AgentType, Tool, initialize_agent
from langchain_openai import OpenAI

In [13]:
tools = [
    Tool(
        name="Sonnets QA System",
        func=state_of_union.run,
        description="useful for when you need to answer questions about Shakespeare's sonnets. Input should be a fully formed question.",
    ),
    Tool(
        name="Ruff QA System",
        func=ruff.run,
        description="useful for when you need to answer questions about ruff (a python linter). Input should be a fully formed question.",
    ),
]

In [14]:
# Construct the agent. We will use the default agent type here.
# See documentation for a full list of options.
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

<ipython-input-14-5458be099bf6>:3: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(


In [15]:
agent.invoke(
    'Given the theme "love", which sonnet best matches this topic and why?'
)



> Entering new AgentExecutor chain...
 I should use the Sonnets QA System to find the best match.
Action: Sonnets QA System
Action Input: "Which sonnet has the theme of love?"
Observation:  The sonnet that has the theme of love is Sonnet 30.
Thought: I should use the Sonnets QA System to find out why Sonnet 30 is the best match.
Action: Sonnets QA System
Action Input: "Why is Sonnet 30 the best match for the theme of love?"
Observation:  Sonnet 30 is the best match for the theme of love because it explores the enduring nature of love and how it transcends time and change. The speaker reflects on how their love remains constant and unchanging, even as they grow older and face challenges. This theme of eternal love is a central aspect of the concept of love and is beautifully expressed in Sonnet 30.
Thought: I now know the final answer.
Final Answer: Sonnet 30 is the best match for the theme of love because it explores the enduring nature of love and how it transcends time and change.


{'input': 'Given the theme "love", which sonnet best matches this topic and why?',
 'output': 'Sonnet 30 is the best match for the theme of love because it explores the enduring nature of love and how it transcends time and change.'}

In [16]:
# agent.invoke(
#     "What did biden say about ketanji brown jackson in the state of the union address?"
# )

In [17]:
# agent.invoke("Why use ruff over flake8?")

In [18]:
agent.invoke("Can you identify who the speaker is addressing in Sonnet 18?")



> Entering new AgentExecutor chain...
 I should use the Sonnets QA System to answer this question.
Action: Sonnets QA System
Action Input: "Who is the speaker addressing in Sonnet 18?"
Observation:  The speaker is addressing their love and expressing their devotion to them.
Thought: I should now use the Ruff QA System to identify the speaker's tone in Sonnet 18.
Action: Ruff QA System
Action Input: "What is the speaker's tone in Sonnet 18?"
Observation:  I don't know.
Thought: I should use the Sonnets QA System again to find out the answer.
Action: Sonnets QA System
Action Input: "What is the speaker's tone in Sonnet 18?"
Observation:  The speaker's tone in Sonnet 18 is admiring and reverent.
Thought: I now know the final answer.
Final Answer: The speaker is addressing their love and expressing their devotion to them. The speaker's tone in Sonnet 18 is admiring and reverent.

> Finished chain.


{'input': 'Can you identify who the speaker is addressing in Sonnet 18?',
 'output': "The speaker is addressing their love and expressing their devotion to them. The speaker's tone in Sonnet 18 is admiring and reverent."}

## Use the Agent solely as a router

You can also set `return_direct=True` if you intend to use the agent as a router and just want to directly return the result of the RetrievalQAChain.

Notice that in the above examples the agent did some extra work after querying the RetrievalQAChain. You can avoid that and just return the result directly.

In [19]:
tools = [
    Tool(
        name="sonnets QA System",
        func=state_of_union.run,
        description="useful for when you need to answer questions about Shakespeare's sonnets. Input should be a fully formed question.",
        return_direct=True,
    ),
    Tool(
        name="Ruff QA System",
        func=ruff.run,
        description="useful for when you need to answer questions about ruff (a python linter). Input should be a fully formed question.",
        return_direct=True,
    ),
]

In [20]:
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

In [21]:
# agent.invoke(
#     "What did biden say about ketanji brown jackson in the state of the union address?"
# )

In [22]:
# agent.invoke("Why use ruff over flake8?")

In [23]:
agent.invoke("What does Sonnet 18 say about the permanence of beauty?")



> Entering new AgentExecutor chain...
 I should use the Sonnet QA System to find the answer.
Action: Sonnet QA System
Action Input: "What does Sonnet 18 say about the permanence of beauty?"
Observation: Sonnet QA System is not a valid tool, try one of [sonnets QA System, Ruff QA System].
Thought: I should use the sonnets QA System to find the answer.
Action: sonnets QA System
Action Input: "What does Sonnet 18 say about the permanence of beauty?"
Observation:  Sonnet 18 suggests that beauty is temporary and will eventually fade with time, but it can be preserved through art and writing. The speaker compares beauty to a flower that will eventually wither, but through his poetry, he can immortalize the beauty of his subject.


> Finished chain.


{'input': 'What does Sonnet 18 say about the permanence of beauty?',
 'output': ' Sonnet 18 suggests that beauty is temporary and will eventually fade with time, but it can be preserved through art and writing. The speaker compares beauty to a flower that will eventually wither, but through his poetry, he can immortalize the beauty of his subject.'}

In [24]:
agent.invoke("What does Sonnet 18 say about the permanence of beauty?")



> Entering new AgentExecutor chain...
 I should use the Sonnet QA System to find the answer.
Action: Sonnet QA System
Action Input: "What does Sonnet 18 say about the permanence of beauty?"
Observation: Sonnet QA System is not a valid tool, try one of [sonnets QA System, Ruff QA System].
Thought: I should use the sonnets QA System to find the answer.
Action: sonnets QA System
Action Input: "What does Sonnet 18 say about the permanence of beauty?"
Observation:  Sonnet 18 suggests that beauty is temporary and will eventually fade with time, but it can be preserved through art and writing. The speaker compares beauty to a flower that will eventually wither, but through his poetry, he can immortalize the beauty of his subject.


> Finished chain.


{'input': 'What does Sonnet 18 say about the permanence of beauty?',
 'output': ' Sonnet 18 suggests that beauty is temporary and will eventually fade with time, but it can be preserved through art and writing. The speaker compares beauty to a flower that will eventually wither, but through his poetry, he can immortalize the beauty of his subject.'}

## Multi-Hop vector store reasoning

Because vector stores are easily usable as tools in agents, it is easy to use answer multi-hop questions that depend on vector stores using the existing agent framework.

In [25]:
tools = [
    Tool(
        name="Sonnets QA System",
        func=state_of_union.run,
        description="useful for when you need to answer questions about Shakespeare's sonnets. Input should be a fully formed question, not referencing any obscure pronouns from the conversation before.",
    ),
    Tool(
        name="Ruff QA System",
        func=ruff.run,
        description="useful for when you need to answer questions about ruff (a python linter). Input should be a fully formed question, not referencing any obscure pronouns from the conversation before.",
    ),
]

In [26]:
# Construct the agent. We will use the default agent type here.
# See documentation for a full list of options.
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

In [27]:
# agent.invoke(
#     "What tool does ruff use to run over Jupyter Notebooks? Did the president mention that tool in the state of the union?"
# )

In [28]:
agent.invoke(
    "Which sonnet talks about aging or the loss of beauty? Does ruff have any rule that could help enforce clean formatting in such poetic lines?"
)



> Entering new AgentExecutor chain...
 I should use the Sonnets QA System to find the sonnet that talks about aging or the loss of beauty.
Action: Sonnets QA System
Action Input: "Which sonnet talks about aging or the loss of beauty?"
Observation:  Sonnet 2 talks about aging and the loss of beauty.
Thought: I should use the Ruff QA System to check if there are any rules that could help enforce clean formatting in poetic lines.
Action: Ruff QA System
Action Input: "Does ruff have any rule that could help enforce clean formatting in such poetic lines?"
Observation:  No, Ruff does not have any specific rules for enforcing clean formatting in poetic lines. Ruff is primarily designed for linting and formatting Python code, so it may not be the best tool for this specific use case. You may want to consider using a different tool or manually enforcing formatting in your poetic lines.
Thought: I now know the final answer.
Final Answer: Sonnet 2 talks about aging and the loss of beauty, and R

{'input': 'Which sonnet talks about aging or the loss of beauty? Does ruff have any rule that could help enforce clean formatting in such poetic lines?',
 'output': 'Sonnet 2 talks about aging and the loss of beauty, and Ruff does not have any specific rules for enforcing clean formatting in poetic lines.'}